# Community detection in Graphs

We will explore the different community detection algorithms in graphs. 

Not all fo them fit with our problem requirements, we will focus on the algorithms which fit the following requirements
* Directed edges.
* Homogeneous nodes. 
* Possible Weighted relationships (in the future).

The algorithms which fit the requirements are:
* Conductance metric
* K-1 Coloring
* Label Propagation
* Louvain
* Modularity metric
* Modularity Optimization
* Strongly Connected Components
* Weakly Connected Components

We will use an additional approach after embedding the graph, we have use Node2Vector and DeepWalk
* K-Means
* DBSCAN

We will explore each one of them, how they work, their advantages and disadvantages and when to use them. 

Check this documentation to understand and explore the Community Detection algorithms:
https://neo4j.com/docs/graph-data-science/current/algorithms/community/

### Conductance metric 

### K-Means Clustering for Community Detection 
Read the embedding graph and apply clustering with cuML (RAPIDS) to use GPU

In [2]:
import torch

In [3]:
# Read embedding representation
node2vector_embeddings_filepath = 'node2vector_embeddings.pt'
node2vector_embeddings = torch.load(node2vector_embeddings_filepath, weights_only=True)
node2vector_embeddings

tensor([[-0.1104,  0.1179, -0.0875],
        [-0.0621,  0.1049, -0.2224],
        [-0.0053,  0.1343, -0.0652],
        ...,
        [-0.1721,  0.1979, -0.1593],
        [-0.1009,  0.2103, -0.1458],
        [-0.2406,  0.1375, -0.0170]], device='cuda:0')

In [4]:
# Init k-means clustering